In [ ]:
import requests
import json
url = ('https://newsapi.org/v2/everything?'
       #'category=general&'
       'domains=bbc.co.uk&'
       #'country=gb&'
       'pageSize=99&'
       'page=1&'
       'from=2019-05-05&to=2019-06-03&'
       'apiKey=34e73062364d41b180a6f6a9625bff79')
response = requests.get(url)

with open('data/data.json', 'w') as outfile:
    json.dump(response.json(), outfile)

In [ ]:
import pandas as pd

news_df = pd.DataFrame(columns=['Title', 'Date', 'Stems'])

In [ ]:

with open("data/data.json") as f:
        loaded_json = json.load(f)
        for i in range(len(loaded_json["articles"])):
            if loaded_json["articles"][i]["content"] is not None:
                news_df = news_df.append({'Title': loaded_json["articles"][i]["title"], 'Date': loaded_json["articles"][i]["publishedAt"]}, ignore_index=True)
                #print(loaded_json["articles"][i]["title"])
                

In [ ]:
news_df

## Stemming

In [ ]:
# import spacy
# from spacy import displacy
# from collections import Counter
# import en_core_web_sm
# nlp = en_core_web_sm.load()

In [ ]:
import nltk
from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
porter = PorterStemmer()

from nltk.tokenize import sent_tokenize, word_tokenize
def stemSentence(sentence):
    token_words=tokenizer.tokenize(sentence)
    token_words
    stop_words = set(stopwords.words('english'))
    #print(stop_words)
    filter_sentense = [w for w in token_words if not w in stop_words]
    
    stem_sentence=[]
    for word in filter_sentense:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
sentence=news_df.Title[0]
x=stemSentence(sentence)
print(sentence)
print(x)


In [ ]:
for i in news_df.index:
    news_df.Stems[i] = stemSentence(news_df.Title[i])

In [ ]:
wordfreq = []
master_wordlist = []
for i in news_df.index:
    wordlist = news_df.Stems[i].split()

    master_wordlist+= wordlist
#print(master_wordlist)
# print(set(master_wordlist))
# for w in master_wordlist:
#     wordfreq.append(master_wordlist.count(w))

# ww = list(zip(master_wordlist, wordfreq))
# print("Pairs\n" + str(ww[:]))

In [ ]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(zip(wordlist,wordfreq))

In [ ]:
word_freq = wordListToFreqDict(master_wordlist)
# print(word_freq)

In [ ]:
# for x, y in word_freq.items():
#     print(x, y)
# print(word_freq.values())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.io import output_notebook

output_notebook()

In [ ]:
fruits = list(word_freq.keys())
counts = list(word_freq.values())

# sorting the bars means sorting the range factors
sorted_fruits = sorted(fruits, key=lambda x: counts[fruits.index(x)])

p = figure(x_range=sorted_fruits, plot_height=500, plot_width=1400, title="Word frequencies",
           toolbar_location="below")

p.vbar(x=fruits, top=counts, width=0.9)
p.xaxis.major_label_orientation = "vertical"
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_size = "25pt"
p.xaxis.major_label_text_font_size = "10pt"
p.y_range.start = 0

show(p)

## Google trends

In [1]:
import pandas as pd
from pytrends.request import TrendReq
import os  
import numpy as np

In [2]:
#functions
def get_interest_and_region(kw_list):
    if not (os.path.isfile('Data/'+str(kw_list[0])+'.csv')):
        print('Term not downloaded yet, downloading now..')
        pytrend = TrendReq(hl='en-US', tz=360)
        #pytrend = TrendReq(hl='en-US', tz=360, timeout=(10,25), proxies=['https://188.217.7.217',], retries=5, backoff_factor=1)

        search_df = pytrend.get_historical_interest(kw_list, year_start=2014, year_end=2018)

        search_df.to_csv('Data/'+str(kw_list[0])+'.csv')
        term_csv = pd.read_csv('Data/'+str(kw_list[0])+'.csv', parse_dates=['date'])
        print(term_csv.head())
        region_df = pytrend.interest_by_region(resolution='COUNTRY', inc_low_vol=False, inc_geo_code=False)
        region_df.to_csv('Data/'+str(kw_list[0])+'_regions'+'.csv')
        print('term with regional data saved')
        return(term_csv, region_df)
    else:
        print('term has been found..')
        pytrend = TrendReq(hl='en-US', tz=360)   
        term_csv = pd.read_csv('Data/'+str(kw_list[0])+'.csv', parse_dates=['date'])
        region_df = pd.read_csv('Data/'+str(kw_list[0])+'_regions'+'.csv')
        print(term_csv.head())
        return(term_csv, region_df)
        
        
def plot_interest_in_time(df2, kw_list):

    from bokeh.io import show, output_file
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from datetime import datetime as dt
    from bokeh.models import DatetimeTickFormatter

    output_notebook()
    TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
    TOOLTIPS = [
        ('month', '@x{datetime}'),
        ('Total searches', '@y'),
    ]

    p = figure(title="Monthly-wise total search of term - "+str(kw_list[0]), y_axis_type="linear", plot_height = 400,
               tools = TOOLS, tooltips = TOOLTIPS, plot_width = 800)


    p.xaxis.axis_label = 'Month'
    p.yaxis.axis_label = 'Total search'
    #p.circle(2010, temp_df.IncidntNum.min(), size = 10, color = 'red')

    p.line(df2.index, df2[str(kw_list[0])],line_color="purple", line_width = 3)

    p.xaxis.formatter=DatetimeTickFormatter(

            days=["%d %B %Y"],
            months=["%d %B %Y"],
            years=["%d %B %Y"],
        )
    # output_file("line_chart.html", title="Line Chart")
    show(p)

In [3]:
kw_list = ['crisis']
time_df, regions_df = get_interest_and_region(kw_list)

Term not downloaded yet, downloading now..
                 date  crisis  isPartial
0 2015-01-07 00:00:00      74      False
1 2015-01-07 01:00:00      73      False
2 2015-01-07 02:00:00      75      False
3 2015-01-07 03:00:00      71      False
4 2015-01-07 04:00:00      68      False
term with regional data saved


In [4]:
## Downsampling to monthly
df2 = time_df.resample('M', on='date').mean()
print(df2.index)

DatetimeIndex(['2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30',
               '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31',
               '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31',
               '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30',
               '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31',
               '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31',
               '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30',
               '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31',
               '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31',
               '2018-01-31', '2018-02-28'],
              dtype='datetime64[ns]', name='date', freq='M')


In [5]:
plot_interest_in_time(df2,kw_list)

Loading BokehJS ...

In [ ]:
# #add year and month
# term_csv['Year'] = term_csv.date.apply(lambda x: x.year)
# term_csv['Month'] = term_csv.date.apply(lambda x: x.month)
# term_csv.head()

In [ ]:
# temp_df = term_csv.groupby(['Year']).mean().reset_index()

# temp_df